# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 8:30AM,254218,10,MSP217180,"Methapharm, Inc.",Released
1,Jan 4 2023 8:30AM,254218,10,MSP217181,"Methapharm, Inc.",Released
2,Jan 4 2023 8:30AM,254218,10,MSP217183,"Methapharm, Inc.",Released
3,Jan 4 2023 8:30AM,254218,10,MSP217184,"Methapharm, Inc.",Released
4,Jan 4 2023 8:30AM,254218,10,MSP217188,"Methapharm, Inc.",Released
5,Jan 4 2023 8:30AM,254218,10,MSP217189,"Methapharm, Inc.",Released
6,Jan 4 2023 8:30AM,254218,10,MSP217186,"Methapharm, Inc.",Released
7,Jan 4 2023 8:30AM,254218,10,MSP217191,"Methapharm, Inc.",Released
8,Jan 4 2023 8:30AM,254196,10,0086280176,ISDIN Corporation,Released
9,Jan 4 2023 8:30AM,254196,10,0086280186,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,254218,Released,8
40,254219,Executing,1
41,254219,Released,4
42,254220,Released,1
43,254223,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254215,NaN,NaN,7.0
254218,NaN,NaN,8.0
254219,NaN,1.0,4.0
254220,NaN,NaN,1.0
254223,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254104,0.0,0.0,1.0
254108,1.0,0.0,23.0
254124,0.0,1.0,0.0
254136,0.0,0.0,45.0
254137,0.0,0.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254104,0,0,1
254108,1,0,23
254124,0,1,0
254136,0,0,45
254137,0,0,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254104,0,0,1
1,254108,1,0,23
2,254124,0,1,0
3,254136,0,0,45
4,254137,0,0,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254104,,,1
1,254108,1,,23
2,254124,,1,
3,254136,,,45
4,254137,,,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc."
8,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation
25,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc."
28,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC"
35,Jan 4 2023 8:30AM,254207,10,Livs Products
36,Jan 4 2023 8:30AM,254220,10,Methapharm-G
37,Jan 3 2023 4:02PM,254223,19,Eli Lilly and Company
38,Jan 3 2023 3:47PM,254219,12,"Uniderm USA, Inc"
43,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc."
44,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc.",,,8
1,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,,,17
2,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",,,3
3,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC",,,7
4,Jan 4 2023 8:30AM,254207,10,Livs Products,,,1
5,Jan 4 2023 8:30AM,254220,10,Methapharm-G,,,1
6,Jan 3 2023 4:02PM,254223,19,Eli Lilly and Company,,,1
7,Jan 3 2023 3:47PM,254219,12,"Uniderm USA, Inc",,1,4
8,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc.",,,1
9,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc.",8,,
1,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,
2,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3,,
3,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC",7,,
4,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,
5,Jan 4 2023 8:30AM,254220,10,Methapharm-G,1,,
6,Jan 3 2023 4:02PM,254223,19,Eli Lilly and Company,1,,
7,Jan 3 2023 3:47PM,254219,12,"Uniderm USA, Inc",4,1,
8,Jan 3 2023 3:11PM,254210,21,"NBTY Global, Inc.",1,,
9,Jan 3 2023 3:11PM,254209,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc.",8,,
1,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,
2,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3,,
3,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC",7,,
4,Jan 4 2023 8:30AM,254207,10,Livs Products,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc.",8.0,NaN,NaN
1,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,NaN,NaN
2,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3.0,NaN,NaN
3,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC",7.0,NaN,NaN
4,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:30AM,254218,10,"Methapharm, Inc.",8.0,0.0,0.0
1,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,0.0,0.0
2,Jan 4 2023 8:30AM,254211,10,"Citieffe, Inc.",3.0,0.0,0.0
3,Jan 4 2023 8:30AM,254215,10,"Snap Medical Industries, LLC",7.0,0.0,0.0
4,Jan 4 2023 8:30AM,254207,10,Livs Products,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5083383,397.0,1.0,1.0
12,508343,4.0,2.0,0.0
15,1016697,72.0,54.0,0.0
19,2287782,76.0,4.0,0.0
20,508253,2.0,0.0,0.0
21,762563,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5083383,397.0,1.0,1.0
1,12,508343,4.0,2.0,0.0
2,15,1016697,72.0,54.0,0.0
3,19,2287782,76.0,4.0,0.0
4,20,508253,2.0,0.0,0.0
5,21,762563,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,397.0,1.0,1.0
1,12,4.0,2.0,0.0
2,15,72.0,54.0,0.0
3,19,76.0,4.0,0.0
4,20,2.0,0.0,0.0
5,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,397.0
1,12,Released,4.0
2,15,Released,72.0
3,19,Released,76.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,1.0,2.0,54.0,4.0,0.0,1.0
Released,397.0,4.0,72.0,76.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,54.0,4.0,0.0,1.0
2,Released,397.0,4.0,72.0,76.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,1.0,2.0,54.0,4.0,0.0,1.0
2,Released,397.0,4.0,72.0,76.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()